# <center>Encontro Módulo 1 - Case Churn</center>
___

## Conteúdo
* 0 - [Introdução](#intro) <br>
* 4 - [Aquisição dos Dados](#data) <br>
* 5 - [Pré-processamento de dados](#clean) <br>
* 6 - [Análise Exploratória](#eda) <br>
* 7 - [Feature Engineering](#fe) <br>
* 8 - [Construção do Modelo](#model) <br>

<a id="casemod1"></a>

<a id='intro'></a>
## 0. Introdução

Esse notebook será utilizado como apoio ao case apresentado no encontro do final do módulo 1.

O intuito desse notebook é mostrar como fazemos para aplicar o *pipeline* de ciência de dados em um projeto real. Para simplicidade, alguns passos do *pipeline* serão passados de forma bem breve e não exaustiva, apenas para fim de exemplificação. Além disso, utilizaremos de técnicas e conceitos que serão apresentados mais pra frente no nosso curso, então não é esperado que você conheça e entenda 100% do código desse notebook.

Como exercício, a medida que você for avançando no curso, volte para esse notebook e tente verificar o que mais poderia ter sido feito em cada etapa, além de tentar entender aqueles passos que não estão claros para você nesse primeiro momento.

<a id='data'></a>
## 4. Aquisição dos Dados

Os dados utilizados nesse notebook estão no arquivo ```bd_churn.txt```. O código abaixo apenas lê o arquivo em um ```DataFrame```, uma estrutura de dados da biblioteca ```pandas```, que é bastante utilizada em ciência de dados. Essa estrutura trata os dados como tabelas, com linhas e colunas, bem parecido com o excel. No próximo módulo teremos uma aula específica para entender melhor sobre esse pacote.

In [ ]:
import pandas as pd

df_churn = pd.read_csv('bd_churn.txt', sep='\t', decimal='.')
df_churn.head()

In [ ]:
df_churn.info()

<a id='clean'></a>
## 5. Pré-processamento de Dados

Como um primeiro passo após o entendimento básico dos dados temos o tratamento deles. Primeiro vamos remover algumas colunas dos dados e depois faremos um tratamento de exemplo. Existem alguns tratamentos diferentes que podemos fazer, mas aqui vamos focar no tratamento de valores nulos. Esse tratamento pode ser feito substituindo os nulos por outros valores ou retirando as linhas que contém nulos. Nesse caso, seguiremos pelo segundo caminho, utilizando o método ```.dropna()```.


In [ ]:
df_churn = df_churn.drop(["CUST_ID", "START_DT", "END_DT", "INPUT_DT", "ACCT_TYPE", 
                          "CLOSED1", "CLOSED2", "CLOSED3", "DUE_DT1", "DUE_DT2", "DUE_DT3", 
                          "CHARGE1", "CHARGE2", "CHARGE3", "CH_BEG1", "CH_BEG2", "CH_BEG3", 
                          "CH_END1", "CH_END2", "CH_END3", "BILL_ST1", "BILL_ST2", "BILL_ST3", 
                          "LT_PMT1", "LT_PMT2", "LT_PM3", "ADJ1", "ADJ2", "ADJ3", "CR_DT1",
                          "CR_DT2", "CR_DT3", "TEN_RAW"], axis=1)
df_churn = df_churn.dropna()
df_churn.info()

In [ ]:
df_churn.head()

<a id='eda'></a>
## 6. Análise Exploratória

### Análise univariada
Uma das primeiras ações que tomamos ao ler dados em um ```DataFrame``` (df) é pegar algumas características dele como tamanho, tipos de dado, quantidade de nulos e etc. Na célula anterior já utilizamos o método ```.head()``` para ver as primeiras linhas do nosso df. Vamos então utilizar também o método ```.info()``` para ver tamanho e tipos de dados e o ```.describe()``` para obter informações sobre as colunas numéricas.

In [ ]:
df_churn.describe()

### Análises de variáveis

Outra tarefa comum na análise de exploratória é entender a nossa variável de interesse (no caso, a coluna **CHURN_FL**) e como ela se relaciona com as demais.

Vamos começar entendendo a taxa de *churn* na nossa base. Dado que a variável de interesse é uma flag se aquele cliente deu *churn* ou não, a taxa nada mais é do que a média dessa variável. Substitua a lacuna pelo nome da coluna para calcular sua média no código abaixo.

In [ ]:
print('A taxa de churn é:', df_churn["CHURN_FL"].mean())

Agora vamos ver como essa taxa se comporta de acordo com outras variáveis dos nossos dados. Apenas para exemplo, vamos analisar o *churn* contra duas variáveis (uma numérica e outra categórica), mas em um projeto analisaríamos todas. Você pode tentar trocar as variáveis do exemplo por outras e ver se funciona, mas eventualmente será necessário algumas alterações para melhorar a visualização.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(data=df_churn,
            x='CHURN_FL',
            y='AGE',
            palette="colorblind")

In [ ]:
sns.kdeplot(data=df_churn,
            x="AGE",
            hue="CHURN_FL",
            common_norm=False)

In [ ]:
sns.barplot(data=df_churn,
            x="CREDIT_CLASS",
            y="CHURN_FL",
            hue="GENDER")

<a id='fe'></a>
## 7. Feature Engineering



In [ ]:
# seleciona colunas categoricas
object_cols = df_churn.select_dtypes(include='object').columns.values
object_cols

In [ ]:
# mostra valores distintos dessas colunas
for col in object_cols:
  print(col, df_churn[col].unique())

In [ ]:
# cria variaveis dummies
df_churn = pd.concat([df_churn.drop(object_cols, axis=1),
                      pd.get_dummies(df_churn[object_cols], prefix=object_cols)], axis=1)
print(df_churn.shape)
df_churn.head()

In [ ]:
df_churn.columns

<a id='model'></a>
## 8. Construção do Modelo

### Separação de treino e teste

Para começar, vamos separar nossos dados em treino e teste. Essa é uma etapa que deve ser feita em todos os modelos supervisionados que você for fazer daqui pra frente, para garantir a qualidade do modelo. Basicamente, aplicamos as técnicas de *machine learning* na base de treino e verificamos os resultados na base de teste para garantir que não há *overfitting*. O código abaixo separa os dados com a função ```train_test_split``` do ```sklearn```, mantendo a mesma proporção da sua variável resposta em ambos os *datasets*.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_churn.drop("CHURN_FL", axis=1).values
y = df_churn["CHURN_FL"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2404)

print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)

Vamos de fato à criação do modelo agora. Para exemplo, vamos usar 3 técnicas de classificação apresentadas no notebook de introdução ao *machine learning*.

### Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix

tree = DecisionTreeClassifier(max_depth=4)
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)
y_prob_pred_tree = [x[1] for x in tree.predict_proba(X_test)]

print(confusion_matrix(y_test, y_pred_tree))

plt.figure(figsize=(20,12))
plot_tree(tree, fontsize=8)
plt.show()

### Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

y_pred_logreg = logreg.predict(X_test)
y_prob_pred_logreg = [x[1] for x in logreg.predict_proba(X_test)]

print(confusion_matrix(y_test, y_pred_logreg))

### k-vizinhos mais próximos (k-Nearest Neighbors/k-NN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
y_prob_pred_knn = [x[1] for x in knn.predict_proba(X_test)]

print(confusion_matrix(y_test, y_pred_knn))

<a id='eval'></a>
### Avaliação dos Modelos

Já vimos na criação dos modelos uma métrica de avaliação, a matriz de confusão. Porém, essa não é a melhor métrica para avaliação dos modelos de classificação. Entraremos mais a fundo nesse tema mais pra frente, mas trazemos aqui a a curva ROC e a área sob a curva.

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr_tree, tpr_tree, thresholds_tree = roc_curve(y_test, y_prob_pred_tree)
auc_tree = auc(fpr_tree, tpr_tree)
fpr_logreg, tpr_logreg, thresholds_logreg = roc_curve(y_test, y_prob_pred_logreg)
auc_logreg = auc(fpr_logreg, tpr_logreg)
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_prob_pred_knn)
auc_knn = auc(fpr_knn, tpr_knn)

plt.figure()
plt.plot(fpr_tree, tpr_tree, color='blue', label='ROC curve tree (area = %0.2f)' % auc_tree)
plt.plot(fpr_logreg, tpr_logreg, color='red', label='ROC curve logreg (area = %0.2f)' % auc_logreg)
plt.plot(fpr_knn, tpr_knn, color='green', label='ROC curve knn (area = %0.2f)' % auc_knn)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()